In [1]:
from model import *
from train_in_roll import *
from dataloader import *
from utils import *
from collections import OrderedDict

/root/miniconda3/envs/myconda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [108]:
dataset = ImageDataSet(win_size=5, start_date=20091231, end_date=20111231, mode='train', indicators={'MA':[20]}, show_volume=True)
image_set = dataset.generate_images(0.04)

[Load Data] | 2023-04-15 10:59:43 Done | Using  104.305 seconds
TRAIN DataSet Initialized
 	 - Image Size:   (32, 15)
 	 - Time Period:  20091231 - 20111231
 	 - Indicators:   {'MA': [20]}
 	 - Volume Shown: True


Generating Images: 100%|██████████| 2283/2283 [02:59<00:00, 12.72it/s]


In [118]:
TRIAN_VAL_SPLIT_RATIO = 0.7
BATCH_SIZE = 64
train_loader_size = int(len(image_set)*TRIAN_VAL_SPLIT_RATIO)
torch_loader_size = len(image_set) - train_loader_size

train_loader, val_loader = torch.utils.data.random_split(image_set, [train_loader_size, torch_loader_size])
train_loader = torch.utils.data.DataLoader(dataset=train_loader, batch_size=BATCH_SIZE, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=val_loader, batch_size=BATCH_SIZE, shuffle=False)

In [119]:
for i, (input, ret1, ret5, ret20) in enumerate(val_loader):
    input11 = input
    ret11, ret51, ret201 = ret1, ret5, ret20
    break

In [120]:
input11.shape

torch.Size([64, 32, 15])

In [111]:
class CNN5d(nn.Module):
    # input in shape of N*32*15
    # output in shape of N*1
    # with two conv layers
    def __init__(self):
        super(CNN5d, self).__init__()
        self.conv1 = nn.Sequential(OrderedDict([
            ('Conv', nn.Conv2d(1, 64, (5, 3), padding=(2, 1), stride=(1, 1), dilation=(1, 1))), # output size: [N, 64, 32, 15]
            ('ReLU', nn.ReLU()),
            ('Max-Pool', nn.MaxPool2d((2,1))) # output size: [N, 64, 16, 15]
        ]))
        self.conv2 = nn.Sequential(OrderedDict([
            ('Conv', nn.Conv2d(64, 128, (5, 3), padding=(2, 1), stride=(1, 1), dilation=(1, 1))), # output size: [N, 128, 16, 15]
            ('ReLU', nn.ReLU()),
            ('Max-Pool', nn.MaxPool2d((2,1))) # output size: [N, 128, 8, 15]
        ]))

        self.FC = nn.Linear(15360, 2)
        self.Softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # input: N * 32 * 15
        x = x.unsqueeze(1).to(torch.float32)   # output size: [N, 1, 32, 15]
        x = self.conv1(x) # output size: [N, 64, 16, 15]
        x = self.conv2(x) # output size: [N, 128, 8, 15]
        x = self.FC(x.view(x.shape[0], -1)) # output size: [N, 2]
        x = self.Softmax(x)
        
        return x

In [112]:
model5 = CNN5d()
model5

CNN5d(
  (conv1): Sequential(
    (Conv): Conv2d(1, 64, kernel_size=(5, 3), stride=(1, 1), padding=(2, 1))
    (ReLU): ReLU()
    (Max-Pool): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (Conv): Conv2d(64, 128, kernel_size=(5, 3), stride=(1, 1), padding=(2, 1))
    (ReLU): ReLU()
    (Max-Pool): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
  )
  (FC): Linear(in_features=15360, out_features=2, bias=True)
  (Softmax): Softmax(dim=1)
)

In [92]:
conv1 = nn.Conv2d(1, 64, (5, 3), padding=(3, 1), stride=(3, 1), dilation=(2, 1))
conv2 = nn.Conv2d(64, 128, (5, 3), padding=(3, 1), stride=(1, 1), dilation=(1, 1))
conv3 = nn.Conv2d(128, 256, (5, 3), padding=(2, 1), stride=(1, 1), dilation=(1, 1))
fc1 = nn.Linear(46080, 2)

In [24]:
input1.shape

torch.Size([64, 64, 60])

In [62]:
input.unsqueeze(1).shape

torch.Size([64, 1, 64, 60])

In [93]:
out = conv1(input.unsqueeze(1).to(torch.float32))
print(out.shape)

out = F.max_pool2d(out, (2, 1))
print(out.shape)

out = conv2(out)
print(out.shape)

out = F.max_pool2d(out, (2, 1))
print(out.shape)

out = conv3(out)
print(out.shape)

out = F.max_pool2d(out, (2, 1))
print(out.shape)

torch.Size([64, 64, 21, 60])
torch.Size([64, 64, 10, 60])
torch.Size([64, 128, 12, 60])
torch.Size([64, 128, 6, 60])
torch.Size([64, 256, 6, 60])
torch.Size([64, 256, 3, 60])


In [102]:
out.view(out.shape[0], -1).shape

torch.Size([64, 46080])

In [107]:
class CNN20d(nn.Module):
    # input in shape of N*64*60
    # output in shape of N*1
    # with 3 conv layers
    def __init__(self):
        super(CNN20d, self).__init__()
        self.conv1 = nn.Sequential(OrderedDict([
            ('Conv', nn.Conv2d(1, 64, (5, 3), padding=(3, 1), stride=(3, 1), dilation=(2, 1))), # output size: [N, 64, 21, 60]
            ('ReLU', nn.ReLU()),
            ('Max-Pool', nn.MaxPool2d((2,1))) # output size: [N, 64, 10, 60]
        ]))
        self.conv2 = nn.Sequential(OrderedDict([
            ('Conv', nn.Conv2d(64, 128, (5, 3), padding=(3, 1), stride=(1, 1), dilation=(1, 1))), # output size: [N, 128, 12, 60]
            ('ReLU', nn.ReLU()),
            ('Max-Pool', nn.MaxPool2d((2,1))) # output size: [N, 128, 6, 60]
        ]))
        self.conv3 = nn.Sequential(OrderedDict([
            ('Conv', nn.Conv2d(128, 256, (5, 3), padding=(2, 1), stride=(1, 1), dilation=(1, 1))), # output size: [N, 256, 6, 60]
            ('ReLU', nn.ReLU()),
            ('Max-Pool', nn.MaxPool2d((2,1))) # output size: [N, 256, 3, 60]
        ]))

        self.FC = nn.Linear(46080, 2)
        self.Softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # input: N * 64 * 60
        x = x.unsqueeze(1).to(torch.float32)   # output size: [N, 1, 64, 60]
        x = self.conv1(x) # output size: [N, 64, 10, 60]
        x = self.conv2(x) # output size: [N, 128, 6, 60]
        x = self.conv3(x) # output size: [N, 256, 3, 60]
        x = self.FC(x.view(x.shape[0], -1)) # output size: [N, 2]
        x = self.Softmax(x)
        
        return x

In [104]:
model20 = CNN20d()

In [ ]:
roll = [((20091231, 20111231), (20111231, 20121231))]
models = [CNN20d().to('cuda'),CNN20d().to('cuda'),CNN20d().to('cuda')]
train_model(models, train_loader, val_loader, num_epochs=10, learning_rate=0.001, batch_size=32, device='cuda' , weight_decay=0.0)